# Face Recognition using LBPH Face Recognizer

### Steps to execute the program

Software used: Jupyter Notebook/Google colab, Pyhton 3.9.x


Instructions to execute the code:

The first code block will import all the necessary libraries that we need in this project(In case that one does not have these libraries installed in his/her system, the IDE will throw a ModuleNotFoundError which can be resolved by simply installing the required libraries)

When the second block of code which is "Creating the Training data", is executed it pops open a window which will capture 100 images of the face that it finds in front of the camera

The third block of code is written to train the model with that data that it just just captured in the above step. It will take not more than 2-3 seconds to train the model(depending on the specifications of the system that one's using).

The fourth block of code is actually a function which is created to verify the face according to the trained model.

Fifth block of code is an extra hand gesture(show a straight hand and then make a fist) verification that is added to ensure a real person is attempting to acces the system. After the execution of this block the function that we defined in the fourth block will be caleed and the face of the user will be verified according to the threshold of the confidence level that we set.

### Importing the required libraries

In [ ]:
import cv2
import numpy as np
import mediapipe as mp


### Step 1 - Creating training data

In [ ]:
faceClassifier = cv2.CascadeClassifier('F:/LBPH Minor Project/haarcascades/haarcascade_frontalface_default.xml')

def faceExtractor(img):
    grayScaleImage = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faceCaptures = faceClassifier.detectMultiScale(grayScaleImage, 1.3, 5)

    if faceCaptures is ():
        return None
    for (x,y,w,h) in faceCaptures:
        croppedFace = img[y:y+h, x:x+w]
    return croppedFace

cap = cv2.VideoCapture(0)
count = 0

# Collecting 100 samples of the face from webcam input
while True:
    ret, frame = cap.read()
    if faceExtractor(frame) is not None:
        count += 1
        face = cv2.resize(faceExtractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        file_name_path = 'F:/LBPH Minor Project/faces/' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)

    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) == 13 or count == 100: #13 is the Enter Key
        break


cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")

### Step 2 - Training the Model

In [ ]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

# Getting the training data we previously made
data_path = 'F:/LBPH Minor Project/faces/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

# Creating arrays for training data and labels
Training_Data, Labels = [], []

# Opening the training images in our datapath
# Creating a numpy array for training data
for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)

# Creating a numpy array for both training data and labels
Labels = np.asarray(Labels, dtype=np.int32)

# Initializing the facial recognizer
model = cv2.face.LBPHFaceRecognizer_create()

# Training the model 
model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model trained sucessefully")


### Step 3 - Defining a method to verify the face

In [ ]:
def verifyFace():
    faceClassifier = cv2.CascadeClassifier('F:/LBPH Minor Project/haarcascades/haarcascade_frontalface_default.xml')

    def face_detector(img, size=0.5):

        # Convert image to grayscale
        grayScaleImage = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        faceCaptures = faceClassifier.detectMultiScale(grayScaleImage, 1.3, 5)
        if faceCaptures is ():
            return img, []

        for (x,y,w,h) in faceCaptures:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
            roi = img[y:y+h, x:x+w]
            roi = cv2.resize(roi, (200, 200))
        return img, roi


    #Open the Webcam
    cap = cv2.VideoCapture(0)

    while True:

        ret, frame = cap.read()
        image, face = face_detector(frame)
        try:
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

            # Pass face to prediction model
            # "results" comprises of a tuple containing the label and the confidence value
            results = model.predict(face)

            if results[1] < 500:
                confidence = int( 100 * (1 - (results[1])/400) )
                display_string = str(confidence) + '% Confident it is User'

            cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)

            if confidence > 75:
                cv2.putText(image, "Unlocked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
                cv2.imshow('Face Recognition', image )
            else:
                cv2.putText(image, "Locked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
                cv2.imshow('Face Recognition', image )

        except:
            cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.putText(image, "Locked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition', image )
            pass

        if cv2.waitKey(1) == 13:
            break

    cap.release()
    cv2.destroyAllWindows()     

### Step 4 - Gesture verification

In [ ]:
cap = cv2.VideoCapture(0)

mpHands = mp.solutions.hands
hands = mpHands.Hands() 

def getCoordinates(id, h, w):
    cx, cy = lm.x*w, lm.y*h
    cv2.circle(img, (int(cx), int(cy)), 1, (255,255,255), cv2.FILLED)  
    return cx, cy

startFaceRecognition=0

while True:
    success, img = cap.read()
    
    if not success: 
        break
    
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)

    h, w, c = img.shape
    handsup=0
    thumbs_correct=0
    fingers_correct=0
    
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            for id, lm in enumerate(handLms.landmark):
                if id == 0: 
                    __, cy_0 = getCoordinates(0, h, w)
                if id == 10: 
                    __, cy_10 = getCoordinates(10, h, w)
            
                if id == 2:
                    __, cy_2 = getCoordinates(2, h, w)
                if id == 3:
                    __, cy_3 = getCoordinates(3, h, w)
            
                if id == 5: 
                    __, cy_5 = getCoordinates(5, h, w)
                if id == 9: 
                    __, cy_9 = getCoordinates(9, h, w)
                if id == 13: 
                    __, cy_13 = getCoordinates(13, h, w)
                if id == 17: 
                    __, cy_17 = getCoordinates(17, h, w)
                    
                if id == 8: 
                    __, cy_8 = getCoordinates(8, h, w)  
                if id == 12: 
                    __, cy_12 = getCoordinates(12, h, w)
                if id == 16: 
                    __, cy_16 = getCoordinates(16, h, w)
                if id == 20: 
                    __, cy_20 = getCoordinates(20, h, w)
            
            if cy_10 < cy_0:
                handsup=1
            else:
                handsup=0
                    
            if (cy_2 > cy_10 and cy_2 < cy_0) and (cy_3 > cy_10 and cy_3 < cy_0):
                thumbs_correct=1
            else:
                thumbs_correct=0
            
            if (cy_5 < cy_8) and (cy_9 < cy_12) and (cy_13 < cy_16) and (cy_17 < cy_20):
                fingers_correct=1
            else:
                figners_correct=0
            
            if handsup==1 and thumbs_correct==1 and fingers_correct==1 and startFaceRecognition==0:
                startFaceRecognition=1
    cv2.imshow("Image", img)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
    if startFaceRecognition==1:
        break

verifyFace()        

cap.release()
cv2.destroyAllWindows()